# IMPORTADOR DE ESTACIONES DE SERVICIO Y PRECIOS POR ESTACIÓN Y PROMEDIO NACIONAL Y POR ESTADO

Actualización al 1 de septiembre de 2021

https://analyticsboutique.github.io/dashboard-simplificado/

Fuentes de datos: 

https://datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel

http://www.cre.gob.mx/da/PreciosPromedioMensuales.csv


In [1]:
# Dependencies
import pandas as pd
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse
import json
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from datetime import date

import warnings
warnings.filterwarnings('ignore')

### NOTA IMPORTANTE: Cambiar fecha del archivo a procesar

In [2]:
# Modificar diariamente
#Fecha = '2021.08.19' 

today = date.today()

if len(str(today.month)) == 1:
    Fecha = str(today.year) + '.0' + str(today.month) + '.' + str(today.day)
else:
    Fecha = str(today.year) + '.' + str(today.month) + '.' + str(today.day)

# Promedios estatates
#Year = 2021

#Month = 7

# SECCIÓN I:

# Importador de datos de Estaciones de Servicio

In [3]:
# 
#tree = ET.parse('../Data_Raw/places_'+ Fecha +'.xml')

#
var_url = urlopen('https://bit.ly/2V1Z3sm') 
tree = ET.parse(var_url)

#
root = tree.getroot()
#root

In [4]:
# Extraemos los elementos del XML
ID = []
name = []
cre_id = []
lng = []
lat = []

for item in range (0,len(root)):
    ID.append(root[item].attrib['place_id'])
    name.append(root[item][0].text)
    cre_id.append(root[item][1].text)
    lng.append(root[item][2][0].text)
    lat.append(root[item][2][1].text)

In [5]:
# Imprime el número de registros
print(len(ID),len(name),len(cre_id),len(lat),len(lng))

14110 14110 14110 14110 14110


In [6]:
# Convertimos a DATAFRAME
Gasolinerias = pd.DataFrame({ 'ID' : ID, 'name' : name, 'cre_id' : cre_id, 'lat' : lat, 'lng' : lng })
Gasolinerias.head()

,ID,name,cre_id,lat,lng
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977


In [7]:
# Guardamos en CSV
Gasolinerias.to_csv('../Data_CSV/places_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', index = False)

# Importador de datos de Precios de Gasolina en Estaciones de Servicio

In [8]:
# 
#tree = ET.parse('../Data_Raw/prices_'+ Fecha +'.xml')

#
var_url = urlopen('https://bit.ly/2JNcTha') 
tree = ET.parse(var_url)

#
root = tree.getroot()
#root

In [9]:
# Extraemos los elementos del XML
ID = []
tipo = []
gas_price = []

for item in range (0,len(root)):
    ID.append(root[item].attrib['place_id'])
    tipo.append(root[item][0].attrib['type']) 
    gas_price.append(root[item].findtext('gas_price'))

In [10]:
# Imprime el número de registros
print(len(ID),len(tipo),len(gas_price))

20646 20646 20646


In [11]:
# Convertimos a DATAFRAME
Precios = pd.DataFrame({ 'ID' : ID, 'tipo' : tipo, 'gas_price' : gas_price })
Precios.head()

,ID,tipo,gas_price
0,11703,regular,22.68
1,11702,premium,27.1
2,11702,regular,24.5
3,11701,regular,18.62
4,11700,regular,23.75


In [12]:
# Guardamos en CSV
Precios.to_csv('../Data_CSV/precios_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', index = False)

# Juntamos datos de Estaciones de Servicio, Precios de Gasolina y Franquicia

In [13]:
# Selecccionamos por tipo de gasolina
Precios_regular = Precios[Precios['tipo'] == 'regular']
Precios_regular.head()

,ID,tipo,gas_price
0,11703,regular,22.68
2,11702,regular,24.5
3,11701,regular,18.62
4,11700,regular,23.75
5,11699,regular,24.25


In [14]:
# Selecccionamos por tipo de gasolina
Precios_premium = Precios[Precios['tipo'] == 'premium']
Precios_premium.head()

,ID,tipo,gas_price
1,11702,premium,27.1
9,11696,premium,24.3
16,3153,premium,24.69
19,11692,premium,24.8
26,11395,premium,25.89


In [15]:
# Selecccionamos por tipo de gasolina
Precios_diesel = Precios[Precios['tipo'] == 'diesel']
Precios_diesel.head()

,ID,tipo,gas_price
8,11697,diesel,25.99
11,11696,diesel,24.98
13,11695,diesel,26.19
17,3153,diesel,24.99
20,11692,diesel,25.99


In [16]:
# Juntamos datos:

# Regular
Gasolinerias_Precios = Gasolinerias

Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_regular, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'regular'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,22.99
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,24.09
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,18.99
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,23.29
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,19.99


In [17]:
# Premium
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_premium, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'premium'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular', 'premium']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,22.99,26.49
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,24.09,25.89
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,18.99,20.99
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,23.29,NaN
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,19.99,22.99


In [18]:
# Regular
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Precios_diesel, on = 'ID', how='outer')

Gasolinerias_Precios = Gasolinerias_Precios.rename(columns={'gas_price': 'diesel'})

Gasolinerias_Precios = Gasolinerias_Precios[['ID', 'name', 'cre_id', 'lat', 'lng', 'regular', 'premium', 'diesel']]

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,22.99,26.49,24.99
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,24.09,25.89,25.35
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,18.99,20.99,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,23.29,NaN,NaN
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,19.99,22.99,NaN


In [19]:
# Is NA's y revisar

Gasolinerias_Precios[Gasolinerias_Precios['cre_id'].isnull()]

,ID,name,cre_id,lat,lng,regular,premium,diesel


In [20]:
# Lectura de la base de franquicias:
Gasolinerias_Franquicias = pd.read_csv('../Data_CSV/Base_Franquicias_2020.10.08.csv')

Gasolinerias_Franquicias = Gasolinerias_Franquicias.rename(columns={'ID_CRE': 'ID'})

Gasolinerias_Franquicias.head()

,ID,Franquicia_Marca
0,2039,PEMEX
1,2040,PERC
2,2041,PEMEX
3,2042,Combu-Express
4,2043,Petro Seven


In [21]:
# Convertimos ID en número
Gasolinerias_Precios['ID'] = pd.to_numeric(Gasolinerias_Precios['ID'])

Gasolinerias_Precios.dtypes

ID          int64
name       object
cre_id     object
lat        object
lng        object
regular    object
premium    object
diesel     object
dtype: object

In [22]:
# Verificamos el tipo de dato del ID
Gasolinerias_Franquicias.dtypes

ID                   int64
Franquicia_Marca    object
dtype: object

In [23]:
# Unión con la base de precios:
Gasolinerias_Precios = pd.merge(Gasolinerias_Precios, Gasolinerias_Franquicias, on = 'ID', how='outer')

Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,22.99,26.49,24.99,PEMEX
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,24.09,25.89,25.35,PERC
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,18.99,20.99,NaN,PEMEX
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,23.29,NaN,NaN,Combu-Express
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,19.99,22.99,NaN,Petro Seven


In [24]:
# Is NA's y revisar

Gasolinerias_Precios[Gasolinerias_Precios['ID'].isnull()]

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca


In [25]:
Gasolinerias_Precios.head()

,ID,name,cre_id,lat,lng,regular,premium,diesel,Franquicia_Marca
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.9214,22.99,26.49,24.99,PEMEX
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.3037,-99.74484,24.09,25.89,25.35,PERC
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.4514,18.99,20.99,NaN,PEMEX
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.3042,23.29,NaN,NaN,Combu-Express
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,19.99,22.99,NaN,Petro Seven


In [26]:
# Guardamos en CSV
Gasolinerias_Precios.to_csv('../Data_CSV/places_&_prices_'+ Fecha +'.csv', sep = ',', encoding = 'utf-8-sig', 
                            index = False)

In [27]:
# Guardamos en CSV
Gasolinerias_Precios.to_json('Data/Price_Stations.json', orient='table', date_format = 'iso', index = False)

# SECCIÓN II:

# Importador de datos de Estaciones de Servicio

### NOTA: Modificar el nombre del archivo para actualizar datos (1 vez cada mes)

http://transparenciacre.westcentralus.cloudapp.azure.com/PNT/73/III/E/PL/Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio.xlsx

In [28]:
#
PreciosPromedioMensuales = pd.read_excel( 'http://transparenciacre.westcentralus.cloudapp.azure.com/PNT/73/III/E/PL/Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio.xlsx', 
                                          sheet_name = 'Cuadro 1.1', skiprows = 3 )

#PreciosPromedioMensuales = pd.read_csv('http://www.cre.gob.mx/da/PreciosPromedioMensuales.csv', encoding = 'latin-1', 
#                              skiprows = 1)

PreciosPromedioMensuales.tail()

,Fecha,Gasolina Regular,Gasolina Premium,Diésel,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
2825,2024-09-26 00:00:00,23.805024,25.238545,25.410330,NaN,NaN,NaN,NaT
2826,2024-09-27 00:00:00,23.796020,25.230838,25.400873,NaN,NaN,NaN,NaT
2827,2024-09-28 00:00:00,23.786586,25.214503,25.389324,NaN,NaN,NaN,NaT
2828,2024-09-29 00:00:00,23.774368,25.211004,25.381932,NaN,NaN,NaN,NaT
2829,2024-09-30 00:00:00,23.769499,25.205455,25.366465,NaN,NaN,NaN,NaT


In [29]:
# Seleccionamos datos de la serie de precios promedio diario
Serie_Precios = PreciosPromedioMensuales[[ 'Fecha', 'Gasolina Regular', 'Gasolina Premium', 'Diésel']]

Serie_Precios = Serie_Precios.rename(columns = { 'Gasolina Regular':'Gas87', 'Gasolina Premium':'Gas91', 'Diésel':'Diesel' })

Serie_Precios = Serie_Precios.dropna()


# Seleccionamos datos de la serie de precios promedio diario
#Serie_Precios = PreciosPromedioMensuales[['FechaCalendario', 'Gasolina mínimo 87 octanos.1', 
#                                          'Gasolina mínimo 91 octanos.1', 'Diésel.1']]

#Serie_Precios = Serie_Precios.rename(columns={ 'FechaCalendario':'Fecha', 'Gasolina mínimo 87 octanos.1':'Gas87',
#                                                  'Gasolina mínimo 91 octanos.1':'Gas91', 'Diésel.1':'Diesel' })

#Serie_Precios = Serie_Precios.dropna()

#

Serie_Precios.head()

,Fecha,Gas87,Gas91,Diesel
0,01/01/2017,15.992677,17.793279,17.047351
1,02/01/2017,16.015220,17.815544,17.077071
2,03/01/2017,16.017609,17.830061,17.076453
3,04/01/2017,16.015294,17.827660,17.077353
4,05/01/2017,16.018897,17.831967,17.081568


In [30]:
# Convert to Datetime
Serie_Precios["Fecha"] = pd.to_datetime( Serie_Precios["Fecha"], format = '%d/%m/%Y' )
Serie_Precios = Serie_Precios.sort_values(by = 'Fecha', ascending = True).reset_index(drop = True)

In [31]:
#
Serie_Precios.tail()

,Fecha,Gas87,Gas91,Diesel
2825,2024-09-26,23.805024,25.238545,25.410330
2826,2024-09-27,23.796020,25.230838,25.400873
2827,2024-09-28,23.786586,25.214503,25.389324
2828,2024-09-29,23.774368,25.211004,25.381932
2829,2024-09-30,23.769499,25.205455,25.366465


In [32]:
# Guardamos en CSV
Serie_Precios.to_csv('../Data_CSV/Price_Time_Serie.csv', sep = ',', encoding = 'utf-8-sig', index = False)

In [33]:
# Guardamos en JSON
Serie_Precios.to_json('Data/Price_Time_Serie.json', orient = 'table', date_format = 'iso', index = False)